# Agregações

Esse script pega o csv que foi salvo no arquivo `coleta-de-dados.ipynb` e cria agregações diversas, que foram usadas para gerar as visualizações de dados.

In [ ]:
import pandas as pd
import json
from datetime import datetime

In [ ]:
now = str(datetime.now())[:-7]
now = now.replace(' ','_')
now = now.replace(":","-")

In [ ]:
df = pd.read_csv("../data/output_files/debate-gazeta/api_output_2018-09-11_11-08-28.csv")

In [ ]:
pd.options.display.max_rows = 100
# RemoveS warning
pd.options.mode.chained_assignment = None

#### 1. Sentimentos positivos e negativos ao longo do tempo
A função cria um dataframe com a soma cumulativa das emoções de cada candidato. Emoções positivas são tratadas como valores positivos. Emoções negativas, como valores negativos. **Esses dados não foram usados na matéria.**

In [ ]:
def cumsum_dominant_feeling(df):
    df = determine_dominant_emotion(df) 

    bad_feelings = ['anger', 'contempt', 'disgust', 'fear', 'sadness']
    df = make_negative(df, bad_feelings)
    df.drop(['neutral', 'blur'], axis=1, inplace=True)

    df["value"] = df.apply(get_value_of_dominant_emotion, axis=1)
    df["cumsum"] = df.value.cumsum()
    df.drop(['index', 'anger', 'contempt', 'disgust', 
             'fear', 'happiness', 'sadness', 'surprise'], axis=1, inplace=True)
    
    return df

In [ ]:
def determine_dominant_emotion(df):
    # Note the abscence of 'neutral' emotion and 'blur'
    cols = ['anger', 'contempt', 'disgust', 'fear',
            'happiness', 'sadness', 'surprise']
    df['dominant_emotion'] = df[cols].idxmax(axis=1)
    return df

In [ ]:
def make_negative(df, bad_feelings):
    for item in bad_feelings:
        df[item] = df[item] * -1
    return df

In [ ]:
def get_value_of_dominant_emotion(row):
    value = row[row.dominant_emotion]

    return pd.Series({
        'value':value
    })

In [ ]:
# Runs function
results = {}
for name in df['name'].unique():
    temp = df[df.name==name]
    results[name] = cumsum_dominant_feeling(temp)
    results[name].to_csv("../data/output_files/debate-tv-gazeta/cumsum_" + name + '_' + now + ".csv", index=False)

#### 2. Qual o percentual de frames para cada emoção?
Essa função determina qual é a emoção dominante, entre as cinco que escolhemos considerar (ver variável `cols`), de cada candidato em cada frame analisado. Depois, ela calcula o percentual do total de frames em que cada emoção foi predominante e salva para um csv.

In [ ]:
def get_dominant_emotion_by_frame(df):
    df = determine_dominant_emotion(df)
    cols = ['happiness','surprise','contempt','anger','sadness']
    
    results = {}
    for name in df['name'].unique():
        values = df[df['name']==name].dominant_emotion.value_counts(normalize=True).to_dict()
        for col in cols:
            if col not in values.keys():
                values[col] = 0
        results[name] = pd.DataFrame(values, index=[name])
        results[name] = results[name][cols] # Reordena
        results[name] = results[name].drop([item for item in results[name].columns if item not in cols], axis=1) # Derruba colunas com valores que queremos ignorar
        # Importante notar que, com esse drop, a soma dos valores de cada candidato não será de 100%.

    results = pd.concat(item for item in results.values())
    results = results.transpose().round(2)
    results.index.name = 'emotion'
    results = results.reset_index()
    return results

In [ ]:
get_dominant_emotion_by_frame(df).to_csv('../data/output_files/debate-gazeta/emotion_by_frame_' + now + '.csv', index=False)

#### 3. A média de cada emoção para cada candidato a cada minuto, calculada usando resample
A função faz um resample da média de emoções demonstradas por cada candidato a cada minuto.

In [ ]:
def get_resampled_average(strftime, df, now):
    cands = df['name'].unique()
    cols = ['anger', 'contempt', 'happiness', 'sadness', 'surprise']
    json_arr = []
    
    # Primeiro, cria um df com a resample de todo o debate
    json_object = {}
    json_object['subject'] = 'total'
    temp = df.copy()
    
    temp['timestamp'] = pd.to_datetime(temp.timestamp, format='%Hh-%Mm-%Ss')
    temp = temp.set_index('timestamp').resample(strftime).mean().fillna(0).drop('index', axis=1).round(2)
    temp = temp.to_dict(orient='list')
    json_object['data'] = []
    for k,v in temp.items():
        if k in cols:
            arr = []
            arr.append(str(k))
            arr.extend(temp[k])
            json_object['data'].append(arr)
        
    json_arr.append(json_object)
    
    # Depois, cria um df com o resample de cada candidato
    for cand in cands:
        json_object = {}
        json_object['subject'] = cand
        temp = df.copy()
            
        temp = df[ df['name'] == cand ]
        temp['timestamp'] = pd.to_datetime(temp.timestamp, format='%Hh-%Mm-%Ss')
        temp = temp.set_index('timestamp').resample(strftime).mean().fillna(0).drop('index', axis=1).round(2)
        temp = temp.to_dict(orient='list')
        json_object['data'] = []
        for k,v in temp.items():
            if k in cols:
                arr = []
                arr.append(str(k))
                arr.extend(temp[k])
                json_object['data'].append(arr)
        
        json_arr.append(json_object)
        
    return json_arr

In [ ]:
# Salva json
with open('../data/output_files/debate-gazeta/resampled_emotions_' + now + '.json', 'w') as output:
    json.dump(get_resampled_average('T', df), output, indent=4)

In [ ]:
# Aqui, salvamos um resample da média para o arquivo csv
def save_csv_resampled_average(strftime, df, now, all_cands):

    if all_cands == True:
        cols = ['timestamp', 'anger', 'contempt', 'happiness', 'sadness', 'surprise', 'neutral']
        temp = df.copy()
        temp['timestamp'] = pd.to_datetime(temp.timestamp, format='%Hh-%Mm-%Ss')
        temp = temp.set_index('timestamp').resample('T').mean().fillna(0).drop('index', axis=1).round(2)
        temp = temp.resample('T').mean().fillna(0).round(2).reset_index()
        temp = temp[cols]
        temp.to_csv("../data/output_files/debate-gazeta/resampled_emotions_all_candidates_2018-09-11_11-09-01.csv", index=False)
    else:
        cols = ['name', 'timestamp', 'anger', 'contempt', 'happiness', 'sadness', 'surprise', 'neutral']
        for cand in df["name"].unique():
            temp = df.copy()
            temp = temp[temp["name"]==cand]
            temp['timestamp'] = pd.to_datetime(temp.timestamp, format='%Hh-%Mm-%Ss')
            temp = temp.set_index('timestamp').resample('T').mean().fillna(0).drop('index', axis=1).round(2)
            temp = temp.resample('T').mean().fillna(0).round(2).reset_index()
            temp["name"] = cand
            temp = temp[cols]
            temp.to_csv("../data/output_files/debate-gazeta/resampled_emotions_" + cand +"_2018-09-11_11-09-01.csv", index=False)

In [ ]:
save_csv_resampled_average('T', df, now, False)
save_csv_resampled_average('T', df, now, True)